# Example 2: Coupled Transmon Qubits

In [2]:
%load_ext autoreload
%autoreload 2
import time

In [3]:
import json
import design as d
import names as n

with open("design_variables.json") as in_file:
    initial_design_variables = json.load(in_file)

from qdesignoptimizer.utils.chip_generation import create_chip_base, ChipType
from qdesignoptimizer.utils.utils import close_ansys

In [4]:
# for this example to store the data in the right place
import os

os.makedirs(os.path.dirname("out/"), exist_ok=True)

## Design assembly

In [5]:
CHIP_NAME = "multi_transmon_chip"
OPEN_GUI = False
chip_type = ChipType(size_x="10mm", size_y="10mm", size_z="-300um")
design, gui = create_chip_base(
    chip_name=CHIP_NAME, chip_type=chip_type, open_gui=OPEN_GUI
)

n.add_design_variables_to_design(design, initial_design_variables)

In [6]:
def render_qiskit_metal_design(design, gui):
    d.add_transmon_plus_resonator(design, group=n.NBR_1)
    d.add_transmon_plus_resonator(design, group=n.NBR_2)

    d.add_coupler(design)

    d.add_route_interconnects(design)

    d.add_launch_pads(design)

    d.add_chargeline(design, group=n.NBR_1)
    d.add_chargeline(design, group=n.NBR_2)
    if gui:
        gui.rebuild()
        gui.autoscale()


render_qiskit_metal_design(design, gui)

## MiniStudies

In [7]:
import mini_studies as ms
import optimization_targets as ot
import parameter_targets as pt
import plot_settings as ps

from qdesignoptimizer.design_analysis import DesignAnalysis, DesignAnalysisState
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qdesignoptimizer.design_analysis_types import MeshingMap

### Single resonator-qubit system
Useful when first tuning up subsystems

In [ ]:
def CoupledLineTee_mesh_names(comp_names):
    all_names_to_mesh = [f"prime_cpw_{comp_names}", f"second_cpw_{comp_names}"]
    return all_names_to_mesh

In [ ]:
MINI_STUDY_GROUP = n.NBR_1
MINI_STUDY = ms.get_mini_study_qb_res(group=MINI_STUDY_GROUP)
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

opt_targets = ot.get_opt_targets_2qubits_resonator_coupler(
    groups=[MINI_STUDY_GROUP],
    opt_target_qubit_freq=True,
    opt_target_qubit_anharm=True,
    opt_target_resonator_freq=True,
    opt_target_resonator_kappa=True,
    opt_target_resonator_qubit_chi=True,
)

In [ ]:
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path="out/" + CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS,
    meshing_map=[
        MeshingMap(component_class=CoupledLineTee, mesh_names=CoupledLineTee_mesh_names)
    ],
)

group_runs = 2  # 10
group_passes = 8  # 6
delta_f = 0.001
for i in range(group_runs):
    design_analysis.update_nbr_passes(group_passes)
    design_analysis.update_delta_f(delta_f)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

In [ ]:
design_analysis.overwrite_parameters()

### Two qubit-resonator system

In [ ]:
MINI_STUDY = ms.get_mini_study_2qb_resonator_coupler()
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

opt_targets = ot.get_opt_targets_2qubits_resonator_coupler(
    groups=[n.NBR_1, n.NBR_2],
    opt_target_qubit_freq=True,
    opt_target_qubit_anharm=True,
    opt_target_resonator_freq=True,
    opt_target_resonator_kappa=False,
    opt_target_resonator_qubit_chi=True,
    opt_target_coupler_freq=True,
)

In [ ]:
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path="out/" + CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS_TWO_QB,
)

group_runs = 2
group_passes = 6
delta_f = 0.001
for i in range(group_runs):
    design_analysis.update_nbr_passes(group_passes)
    design_analysis.update_delta_f(delta_f)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

In [ ]:
design_analysis.overwrite_parameters()

### Purcell decay into charge line

The T1 of the qubit is limited by its decay into the charge line. This ministudy will change the distance of the charge line from the transmon pocket to optimize a T1 limit of 1 ms. This value is computed with simple capacitance matrix simulations and a classical model, assuming that the qubit frequency has already been optimized.

In [ ]:
close_ansys()

In [ ]:
MINI_STUDY_GROUP = 1
MINI_STUDY = ms.get_mini_study_qb_charge_line(group=MINI_STUDY_GROUP)
opt_targets = ot.get_opt_targets_qb_charge_line(group=MINI_STUDY_GROUP)
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

In [ ]:
# %matplotlib inline
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path="out/" + CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS_CHARGE_LINE_DECAY,
)

In [ ]:
group_runs = 5
group_passes_cap = 5

for i in range(group_runs):
    design_analysis.update_nbr_passes_capacitance_ministudies(group_passes_cap)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

### Optimize capacitance target

In [ ]:
MINI_STUDY_BRANCH = 1
MINI_STUDY = ms.get_mini_study_resonator_capacitance(group=MINI_STUDY_BRANCH)
opt_targets = ot.get_opt_target_capacitance(group=MINI_STUDY_BRANCH)

RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)
# %matplotlib inline
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path="out/" + CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS_CAPACITANCE,
)

In [ ]:
group_runs = 1
group_passes_cap = 10

for i in range(group_runs):
    design_analysis.update_nbr_passes_capacitance_ministudies(group_passes_cap)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

### optimise scattering analysis and eigenmode

In [8]:
MINI_STUDY_BRANCH = 1
MINI_STUDY = ms.get_mini_study_scattering_analysis(group=MINI_STUDY_BRANCH)
opt_targets = ot.get_opt_target_scattering_analysis(group=MINI_STUDY_BRANCH)

RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)
# %matplotlib inline
design_analysis_state = DesignAnalysisState(
    
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path="out/" + CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_design_variables=False,
    plot_settings=None,
    
)

[INFO|2025-03-11 06:58:48]: self.eig_solver.sim.setup 
{
    "name": "Resonator_setup",
    "reuse_selected_design": true,
    "reuse_setup": true,
    "min_freq_ghz": 1,
    "n_modes": 1,
    "max_delta_f": 0.5,
    "max_passes": 10,
    "min_passes": 1,
    "min_converged": 1,
    "pct_refinement": 30,
    "basis_order": 1,
    "vars": {
        "Lj": "10 nH",
        "Cj": "0 fF"
    }
}


pyaedt INFO: using existing logger.
pyaedt INFO: Launching PyAEDT outside AEDT with CPython and PythonNET.
pyaedt INFO: AEDT installation Path C:\Program Files\AnsysEM\AnsysEM21.2\Win64.
pyaedt INFO: Launching AEDT with module PythonNET.
pyaedt INFO: Ansoft.ElectronicsDesktop.2021.2 Started with process ID 21888.
pyaedt INFO: pyaedt v0.6.46
pyaedt INFO: Python version 3.10.0 | packaged by conda-forge | (default, Nov 20 2021, 02:18:13) [MSC v.1916 64 bit (AMD64)]
pyaedt INFO: Project Project196 has been created.
pyaedt INFO: No design is present. Inserting a new design.
pyaedt INFO: Added design 'HFSS_LJH' of type HFSS.
pyaedt INFO: Aedt Objects initialized
pyaedt INFO: Variable Manager initialized
pyaedt INFO: Design Loaded
pyaedt INFO: Successfully loaded project materials !
pyaedt INFO: Materials Loaded


[INFO|2025-03-11 06:59:06]: self.eig_solver.sim.setup 
{
    "name": "Resonator_setup",
    "reuse_selected_design": true,
    "reuse_setup": true,
    "min_freq_ghz": 1,
    "n_modes": 1,
    "max_delta_f": 0.5,
    "max_passes": 10,
    "min_passes": 1,
    "min_converged": 1,
    "pct_refinement": 30,
    "basis_order": 1,
    "vars": {
        "Lj": "10 nH",
        "Cj": "0 fF"
    }
}
INFO 06:59AM [connect_project]: Connecting to Ansys Desktop API...
INFO 06:59AM [load_ansys_project]: 	Opened Ansys App
INFO 06:59AM [load_ansys_project]: 	Opened Ansys Desktop v2021.2.0
INFO 06:59AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/nirpit/Documents/Ansoft/
	Project:   Project196
INFO 06:59AM [connect_design]: 	Opened active design
	Design:    HFSS_LJH [Solution type: DrivenModal]
WARNING 06:59AM [connect_setup]: 	No design setup detected.
WARNING 06:59AM [connect_setup]: 	Creating driven modal default setup.
INFO 06:59AM [get_setup]: 	Opened setup `Setup`  (<class 'p

In [9]:
group_runs = 1
group_passes = 10
delta_f = 0.001
for i in range(group_runs):
    design_analysis.update_nbr_passes(group_passes)
    design_analysis.update_delta_f(delta_f)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

design_analysis.system_optimized_params

[INFO|2025-03-11 06:59:08]: Updated_design_vars
{
    "cpw_width": "10 um",
    "cpw_gap": "6 um",
    "design_var_width_qubit_1": "400um",
    "design_var_width_qubit_2": "400um",
    "design_var_cl_pos_x_qubit_1": "-2600um",
    "design_var_cl_pos_y_qubit_1": "-1800um",
    "design_var_cl_pos_x_qubit_2": "-2600um",
    "design_var_cl_pos_y_qubit_2": "1800um",
    "design_var_length_resonator_1_capacitance": "20um",
    "design_var_length_resonator_2_capacitance": "20um",
    "design_var_lj_qubit_1": "12.1nH",
    "design_var_lj_qubit_2": "9.1nH",
    "design_var_cj_qubit_1": "0fF",
    "design_var_cj_qubit_2": "0fF",
    "design_var_length_resonator_1": "7500um",
    "design_var_length_resonator_2": "6500um",
    "design_var_length_coupler_1to2": "4000um",
    "design_var_coupl_length_qubit_1_resonator_1": "100um",
    "design_var_coupl_length_qubit_2_resonator_2": "100um",
    "design_var_coupl_length_resonator_1_tee": "170um",
    "design_var_coupl_length_resonator_2_tee": "160um"


pyaedt ERROR: Error in :   File "c:\Users\nirpit\AppData\Local\anaconda3\envs\qdesign-env\lib\site-packages\pyaedt\application\Design.py", line 2851, in delete_design
pyaedt ERROR: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147024371), None) - delete_design -  GENERAL OR AEDT ERROR.
pyaedt ERROR: Project: Project196, Design: ('get_mini_study_scattering_analysis',) (Eigenmode), [error] Script macro error: Unable to locate Design: ('get_mini_study_scattering_analysis',)_drivenmodal (6:59:09 am  Mar 11, 2025)

pyaedt ERROR: Arguments with values: 
pyaedt ERROR:     name = ('get_mini_study_scattering_analysis',)_drivenmodal 
pyaedt ERROR:     fallback_design = None 
pyaedt ERROR: Check Online documentation on: https://aedt.docs.pyansys.com/search.html?q=delete_design
pyaedt INFO: Project Project196 Saved correctly


[INFO|2025-03-11 06:59:11]: self.eig_solver.sim.setup 
{
    "name": "Resonator_setup",
    "reuse_selected_design": true,
    "reuse_setup": true,
    "min_freq_ghz": 1,
    "n_modes": 1,
    "max_delta_f": 0.5,
    "max_passes": 10,
    "min_passes": 1,
    "min_converged": 1,
    "pct_refinement": 30,
    "basis_order": 1,
    "vars": {
        "Lj": "10 nH",
        "Cj": "0 fF"
    }
}
INFO 06:59AM [connect_project]: Connecting to Ansys Desktop API...
INFO 06:59AM [load_ansys_project]: 	Opened Ansys App
INFO 06:59AM [load_ansys_project]: 	Opened Ansys Desktop v2021.2.0
INFO 06:59AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/nirpit/Documents/Ansoft/
	Project:   Project196
INFO 06:59AM [connect_design]: 	Opened active design
	Design:    HFSS_LJH [Solution type: DrivenModal]
INFO 06:59AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssDMSetup'>)
INFO 06:59AM [connect]: 	Connected to project "Project196" and design "HFSS_LJH" 😀 

06:59AM 12s WARNING 

Design "('get_mini_study_scattering_analysis',)" info:
	# eigenmodes    1
	# variations    1
Design "('get_mini_study_scattering_analysis',)" info:
	# eigenmodes    1
	# variations    1


{'qubit_1_freq': None,
 'qubit_1_purcell_limit_T1': None,
 'resonator_1_freq': 7665930840.849999,
 'resonator_1_kappa': 131734.6740694,
 'qubit_2_freq': None,
 'resonator_2_freq': None,
 'resonator_2_kappa': None,
 'coupler_1to2_freq': None,
 'qubit_1_to_qubit_1_nonlin': None,
 'qubit_1_to_resonator_1_nonlin': None,
 'qubit_2_to_qubit_2_nonlin': None,
 'qubit_2_to_resonator_2_nonlin': None,
 'prime_cpw_name_tee1_to_second_cpw_name_tee1_capacitance': None}

In [19]:
from qdesignoptimizer.design_analysis_types import MiniStudy, ScatteringStudy
group = 1
resonator = [n.RESONATOR_1, n.RESONATOR_2][group - 1]
qiskit_component_names = [n.name_mode(resonator), n.name_tee(group)]
Scatterstudy = ScatteringStudy(nbr_passes = 40,
                max_delta_s =0.005,
                basis_order = 1,
                solution_type = "fast",
                qiskit_component_names=qiskit_component_names,
                port_list=[
                    (n.name_tee(group), "prime_end", 50),
                    (n.name_tee(group), "prime_start", 50),
                ],
                open_pins=[(n.name_mode(resonator), "start")],
                mode=[resonator],
                freq_span_ghz = 100e6/1e9,)

In [20]:
probe_freq, Sij = design_analysis.run_decay(Scatterstudy)

INFO 11:17AM [connect_project]: Connecting to Ansys Desktop API...
INFO 11:17AM [load_ansys_project]: 	Opened Ansys App
INFO 11:17AM [load_ansys_project]: 	Opened Ansys Desktop v2021.2.0
INFO 11:17AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/nirpit/Documents/Ansoft/
	Project:   Project196
INFO 11:17AM [connect_design]: 	Opened active design
	Design:    ('get_mini_study_scattering_analysis',)_drivenmodal [Solution type: DrivenModal]
INFO 11:17AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssDMSetup'>)
INFO 11:17AM [connect]: 	Connected to project "Project196" and design "('get_mini_study_scattering_analysis',)_drivenmodal" 😀 

INFO 11:17AM [connect_design]: 	Opened active design
	Design:    ('get_mini_study_scattering_analysis',)_drivenmodal [Solution type: DrivenModal]
INFO 11:17AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssDMSetup'>)


Running scattering analysis for resonator_1
step_size KHz 1000.0000000000052
7.615930840849999 7.71593084085 100


INFO 11:17AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssDMSetup'>)
INFO 11:17AM [analyze]: Analyzing setup Setup : Sweep4


pinfo_setup
solution_freq default 7.66593084085GHz


LinAlgError: SVD did not converge

In [13]:
design_analysis.kappa_target

1000000.0

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot(probe_freq,Sij):
    real = Sij.real
    imag = Sij.imag
    # Setting a clean style
    plt.style.use('ggplot')

    # Creating subplots with a better layout
    fig, axs = plt.subplots(1,3, figsize=(10,5), constrained_layout=True)

    # Subplot 1: Complex plane (Re vs Im)
    axs[0].plot(real, imag, label='Raw Data', color='blue', linestyle='-', marker='o', markersize=4)

    axs[0].set_xlabel('Re(S21)', fontsize=12)
    axs[0].set_ylabel('Im(S21)', fontsize=12)
    axs[0].legend(fontsize=10)
    axs[0].set_title('Complex Plane', fontsize=14)

    # Subplot 2: Magnitude vs Frequency
    axs[1].plot(probe_freq * 1e-9, np.abs(Sij), label='Raw Data', color='green', linewidth=1.5)

    axs[1].set_xlabel('f (GHz)', fontsize=12)
    axs[1].set_ylabel('|S21|', fontsize=12)
    axs[1].legend(fontsize=10)
    axs[1].set_title('Magnitude vs Frequency', fontsize=14)

    # Subplot 3: Phase vs Frequency
    axs[2].plot(probe_freq * 1e-9, np.angle(Sij), label='Raw Data', color='purple', linewidth=1.5)

    axs[2].set_xlabel('f (GHz)', fontsize=12)
    axs[2].set_ylabel('arg(|S21|)', fontsize=12)
    axs[2].legend(fontsize=10)
    axs[2].set_title('Phase vs Frequency', fontsize=14)


    # Adding a global title
    fig.suptitle('Improved Visualization of S21 Data', fontsize=16)

    # Displaying the plot
    plt.show()

In [ ]:

# plot(probe_freq,Sij)
 
# clip frequency between 7.175 to 7.177 GHz using numpy

clip_freq = probe_freq[np.logical_and(probe_freq >= 7.416e9, probe_freq <= 7.417e9)]
clip_signal = Sij[np.logical_and(probe_freq >= 7.416e9, probe_freq <= 7.417e9)]

plot(clip_freq,clip_signal)



In [ ]:
from resonator_tools import circuit

port = circuit.notch_port()

port.add_data(clip_freq, clip_signal)
port.autofit()
port.plotall()
port.fitresults['fr']/port.fitresults['Ql']

In [ ]:
port.


In [ ]:
port.fitresults['fr']/4965

## View Optimization results

In [ ]:
design_analysis.get_cross_kerr_matrix(iteration=-1)

In [ ]:
design_analysis.get_eigenmode_results()

In [ ]:
close_ansys()

In [ ]:
from resonator_tools import circuit

In [ ]:
port = circuit.notch_port()
